<a href="https://colab.research.google.com/github/ZackPashkin/tensorflow_glasses_classifier_plus_tflite/blob/master/Glasses_TensorFlow_Classifier1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ZackPashkin/tensorflow_glasses_classifier_plus_tflite

Cloning into 'tensorflow_glasses_classifier_plus_tflite'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 2035 (delta 30), reused 24 (delta 23), pack-reused 1996
Receiving objects: 100% (2035/2035), 87.34 MiB | 41.38 MiB/s, done.
Resolving deltas: 100% (829/829), done.


In [0]:
#extract and move images to dataset directory
!unzip -qq tensorflow_glasses_classifier_plus_tflite/with_glasses.zip
!unzip -qq tensorflow_glasses_classifier_plus_tflite/without_glasses.zip
!mkdir tensorflow_glasses_classifier_plus_tflite/tf_files/dataset
!mv with_glasses tensorflow_glasses_classifier_plus_tflite/tf_files/dataset
!mv without_glasses tensorflow_glasses_classifier_plus_tflite/tf_files/dataset

In [0]:
# MobileNet is available 0.25; 0,5; 0.75 and 1.0
# Image size can be 128 or 224
!python -m tensorflow-for-poets-2.scripts.retrain \
  --bottleneck_dir=tensorflow_glasses_classifier_plus_tflite/tf_files/bottlenecks \
  --how_many_training_steps=15000 \
  --model_dir=tensorflow_glasses_classifier_plus_tflite/tf_files/models/ \
  --summaries_dir=tensorflow_glasses_classifier_plus_tflite/tf_files/training_summaries/"mobilenet_0.25_224" \
  --output_graph=tensorflow_glasses_classifier_plus_tflite/tf_files/retrained_graph.pb \
  --output_labels=tensorflow_glasses_classifier_plus_tflite/tf_files/retrained_labels.txt \
  --architecture="mobilenet_0.25_224" \
  --image_dir=tensorflow_glasses_classifier_plus_tflite/tf_files/dataset

>> Downloading mobilenet_v1_0.25_224_frozen.tgz 100.3%
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.6/logging/__init__.py", line 994, in emit
    msg = self.format(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 840, in format
    return fmt.format(record)
  File "/usr/lib/python3.6/logging/__init__.py", line 577, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/tensorflow-for-poets-2/scripts/retrain.py", line 1326, in <module>
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/platform/app.py"

In [0]:
#launch tensorboard
!tensorboard --logdir tensorflow_glasses_classifier_plus_tflite/tf_files/training_summaries &

In [0]:
!pkill -f "tensorboard"

In [0]:
#make prediction
!python -m tensorflow_glasses_classifier_plus_tflite.label_image \
    --graph=tensorflow_glasses_classifier_plus_tflite/tf_files/retrained_graph.pb  \
    --image=tensorflow_glasses_classifier_plus_tflite/tf_files/example_data_glasses/without_glasses/000004.jpg \
    --labels=tensorflow_glasses_classifier_plus_tflite/tf_files/retrained_labels.txt

In [0]:
#Compress model converting to TFlite

#IMAGE_SIZE=224
!tflite_convert \
  --graph_def_file=tensorflow_glasses_classifier_plus_tflite/tf_files/retrained_graph.pb \
  --output_file=tensorflow_glasses_classifier_plus_tflite/tf_files/optimized_graph025_224.lite \
  --input_format=TENSORFLOW_GRAPHDEF \
  --output_format=TFLITE \
  --input_shape=1,224,224,3 \
  --input_array=input \
  --output_array=final_result \
  --inference_type=FLOAT \
  --input_data_type=FLOAT

2018-12-09 21:33:55.655465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-12-09 21:33:55.655945: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-12-09 21:33:55.655991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2018-12-09 21:33:56.066678: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-12-09 21:33:56.066757: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2018-12-09 21:33:56.066787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2018-12-09 21:33:56.067045: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc